In [14]:
pip install pytesseract pillow opencv-python

Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import pytesseract
import cv2
from PIL import Image

# OCR을 수행할 이미지 파일명
image_filename = "5.PNG"
image_path = os.path.abspath(image_filename)

try:
    # 1️⃣ 이미지 불러오기
    img = cv2.imread(image_path)

    # 2️⃣ 이미지 해상도 조정 (1.5배 확대)
    scale_percent = 150
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)

    # 3️⃣ 흑백 변환 & 명암 대비 향상
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)

    # 4️⃣ 노이즈 제거 (Median Blur)
    denoised = cv2.medianBlur(equalized, 3)

    # 5️⃣ 가우시안 블러 후 Otsu 이진화
    blur = cv2.GaussianBlur(denoised, (5, 5), 0)
    _, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 6️⃣ 모폴로지 연산 (Dilation & Erosion)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    morphed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # 7️⃣ OCR 실행 (딥러닝 기반 엔진 사용)
    config = "--oem 1 --psm 6"
    extracted_text = pytesseract.image_to_string(morphed, lang="eng+kor", config=config)

    # 8️⃣ 결과 출력
    print(f"📄 {image_filename} - 추출된 텍스트:")
    print(extracted_text)

except FileNotFoundError:
    print(f"❌ 오류: '{image_filename}' 파일을 찾을 수 없습니다.")

except Exception as e:
    print(f"❌ 오류: {e}")


📄 5.PNG - 추출된 텍스트:
©6®@ —O
가천 헤 커 톤 최 서 연
a}: 2 eos
안녕 내일 몇 시 에 만 날 래 ? ~

오후 9 시 12 분 에 가 천 대 학 교 에서 만 나 는 거 Of

만나서 피시방 가서 를 하자

오조 가 / ME 마 라 탕 먹자

^) 가천 혜 커튼 최 서 연
~ 일 겠어 그 렇 게 하자 og ay
© OPQ 00 전송

